In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [15]:
# linear regression using ordinary least squares 
def linear_regression(x, y):  
    X = np.ones((len(x), 2), np.uint64)
    X[:, 1] = x
    Xt = X.transpose()
    
    # compute beta = (X^t . X)^-1 . X^t . Y
    coefficients = ((np.linalg.inv(Xt.dot(X))).dot(Xt)).dot(y)
    coefficients[0], coefficients[1] = coefficients[1], coefficients[0]
    
    return coefficients

# format defence and hitpoint experience
def process_df(df):
    exp_type = np.zeros(len(df))
    
    d1 = { "Time (min)" : df["Time (min)"],
           "Exp Gained" : df["Def Exp Gained"],
           "Exp Type"   : exp_type } # initialize exp type column
    defence_df = pd.DataFrame(data = d1)
    defence_df.iloc[:, 2] = "defence" # declare exp type column

    d2 = { "Time (min)" : df["Time (min)"],
           "Exp Gained" : df["HP Exp Gained"],
           "Exp Type"   : exp_type }
    hitpoint_df = pd.DataFrame(data = d2)
    hitpoint_df.iloc[:, 2] = "hitpoint"

    processed_df = defence_df.append(hitpoint_df)

    return processed_df

# return the Y values needed for linear regression 
def get_linreg_Y(df):
    section_length = len(df) // 2
    Y = df.iloc[section_length:, 1].values
    
    return Y

# return a dataframe containing copies of dataframes 1 and 2 - the Event RPG dataframe must go first
def combine_df(df1, df2):
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    
    df1_copy["Weapon"], df2_copy["Weapon"] = np.zeros(len(df1)), np.zeros(len(df1))
    df1_copy.loc[:, "Weapon"] = "Event RPG"
    df2_copy.loc[:, "Weapon"] = "Iron Scimitar"
    
    combined_df = df1_copy.append(df2_copy)
    
    return combined_df  

# calculate the number of hours required to reach final_exp given the initial_exp and coefficients
def extrapolate_time(initial_exp, final_exp, coefficients):
    exp_required = final_exp - initial_exp
    
    time = round((exp_required - coefficients[1]) / coefficients[0] / 60)
    
    return time

def display_coefficients(exp_type, coefficients, linregress_results):
    print(exp_type)
    print("Slope and x-intercept using matricies:", coefficients)
    print("Slope and x-intercept using SciPy stats:", linregress_results[0:2])
    
def display_correlation_coefficient(exp_type, linregress_results):
    print("The", exp_type, "correlation coefficient is", linregress_results[2])
    
def display_time_required(weapon, time_required):
    print(time_required, "hours are needed to reach level 75 Defence using the", weapon)  

In [3]:
# load preprocessed data
rpg_df = pd.read_csv("data/rpg_data.csv")
scim_df = pd.read_csv("data/scim_data.csv")

In [4]:
# process data
processed_rpg_df = process_df(rpg_df)
processed_scim_df = process_df(scim_df)
combined_df = combine_df(processed_rpg_df, processed_scim_df)

In [5]:
# write processed data to csv
combined_df.to_csv("data/combined_data.csv", index = False)

In [6]:
# load processed data
combined_df = pd.read_csv("data/combined_data.csv")

In [7]:
# transform data for linear regression
section_length = len(combined_df) // 4
time = np.arange(0, 190, 10)

# sections
s = np.arange(section_length, 5 * section_length, section_length)

rpg_defence_exp, rpg_hitpoint_exp = combined_df.iloc[:s[0], 1], combined_df.iloc[s[0]:s[1], 1]
scim_defence_exp, scim_hitpoint_exp = combined_df.iloc[s[1]:s[2], 1], combined_df.iloc[s[2]:s[3], 1]

In [16]:
# linear regression coefficients for the Event RPG
rpg_defence_coefficients = linear_regression(time, rpg_defence_exp.values)
rpg_defence_linregress_results = stats.linregress(time, rpg_defence_exp.values)
rpg_hitpoint_coefficients = linear_regression(time, rpg_hitpoint_exp.values)
rpg_hitpoint_linregress_results = stats.linregress(time, rpg_hitpoint_exp.values)

print("--- Event RPG ---")
display_coefficients("Defence EXP", rpg_defence_coefficients, rpg_defence_linregress_results)
display_coefficients("Hitpoint EXP", rpg_hitpoint_coefficients, rpg_hitpoint_linregress_results)

display_correlation_coefficient("Defence EXP", rpg_defence_linregress_results)

--- Event RPG ---

 Defence EXP
Slope and x-intercept using matricies: [25.31087719 -3.87368421]
Slope and x-intercept using SciPy stats: (25.310877192982453, -3.873684210526335)

 Hitpoint EXP
Slope and x-intercept using matricies: [ 8.41586667 -1.288     ]
Slope and x-intercept using SciPy stats: (8.415866666666666, -1.2879999999998972)
The Defence EXP correlation coefficient is 0.9998153806149918


In [ ]:
# linear regression coefficients for the iron scimitar
scim_defence_coefficients = linear_regression(time, scim_defence_exp.values)
scim_defence_linregress_results = stats.linregress(time, scim_defence_exp.values)
scim_hitpoint_coefficients = linear_regression(time, scim_hitpoint_exp.values)
scim_hitpoint_linregress_results = stats.linregress(time, scim_hitpoint_exp.values)

print("--- Iron Scimitar ---")
display_coefficients("Defence EXP", scim_defence_coefficients, scim_defence_linregress_results)
display_coefficients("Hitpoint EXP", scim_hitpoint_coefficients, scim_hitpoint_linregress_results)

display_correlation_coefficient(scim_defence_linregress_results)

In [ ]:
# visualize data
exp_scatter = sns.relplot(x = "Time (min)", 
                          y = "Exp Gained",
                          hue = "Exp Type", 
                          col = "Weapon",
                          data = combined_df)

exp_scatter.set(xlabel = "Time (min)",
                ylabel = "Experience",
                yticks = range(0, 5000, 500))

plt.show()

In [ ]:
# extrapolate to obtain the amount of time required to reach level 75 defence (1210421 exp) from 133685 exp
rpg_time_required = extrapolate_time(133685, 1210421, rpg_defence_coefficients)
display_time_required("Event RPG", rpg_time_required)

scim_time_required = extrapolate_time(133685, 1210421, scim_defence_coefficients)
display_time_required("Iron Scimitar", scim_time_required)